# 如何缓存昂贵的节点

<div class="警告提示">
<p class="admonition-title">先决条件</p>
<p>
本指南假设您熟悉以下内容：
<ul>
<li>
<a href="/langgraphjs/concepts/low_level/#graphs">
图表
</a>
</li>
<li>
<a href="/langgraphjs/concepts/low_level/#nodes">
节点
</a>
</li>
</ul>
</p>
</div>

当您想要避免重复操作时，例如在执行昂贵的操作（无论是时间还是成本）时，节点缓存非常有用。LangGraph 允许您向图表中的节点添加个性化的缓存策略。

要配置缓存策略，请将 `cachePolicy` 参数传递给 `addNode` 方法。在以下示例中，我们指定生存时间 (TTL) 为 120 秒的缓存策略和默认密钥序列化功能。然后，要为图启用节点级缓存，请在编译图时设置 `cache` 参数。下面的示例使用 `InMemoryCache` 设置带有内存缓存的图表。

In [4]:
import { StateGraph, Annotation, START } from "@langchain/langgraph";
import { InMemoryCache } from "@langchain/langgraph-checkpoint";

const StateAnnotation = Annotation.Root({
  items: Annotation<string[]>({
    default: () => [],
    reducer: (acc, item) => [...acc, ...item],
  }),
});

const cache = new InMemoryCache();

const graph = new StateGraph(StateAnnotation)
  .addNode(
    "node",
    async () => {
      // 模拟昂贵的操作
      await new Promise((resolve) => setTimeout(resolve, 3000));
      return { items: ["Hello, how are you?"] };
    },
    { cachePolicy: { ttl: 120 } }
  )
  .addEdge(START, "node")
  .compile({ cache });


由于缓存为空，初始运行将花费 3 秒。使用相同输入的后续运行将被缓存并立即生成。

In [5]:
console.time("First run");
await graph.invoke({ items: ["Hello!"] });
console.timeEnd("First run");

console.time("Second run");
await graph.invoke({ items: ["Hello!"] });
console.timeEnd("Second run");


First run: 3.006s
Second run: 4.148ms


您还可以将自定义密钥序列化函数传递给 `cachePolicy` 参数。这可用于跳过序列化中的某些字段，例如消息 ID，这些字段在每次运行时可能是随机的。

In [6]:
import { StateGraph, MessagesAnnotation, START } from "@langchain/langgraph";
import { InMemoryCache } from "@langchain/langgraph-checkpoint";
import { BaseMessage } from "@langchain/core/messages";

const cache = new InMemoryCache();
const graph = new StateGraph(MessagesAnnotation)
  .addNode(
    "node",
    async () => {
      await new Promise((resolve) => setTimeout(resolve, 3000));
      return { messages: [{ type: "ai", content: "Hello, how are you?" }] };
    },
    {
      cachePolicy: {
        ttl: 120,
        keyFunc([{ messages }]: [{ messages: BaseMessage[] }]) {
          // 根据消息的内容和相对位置进行缓存
          return JSON.stringify(messages.map((m, idx) => [idx, m.content]));
        },
      },
    }
  )
  .addEdge(START, "node")
  .compile({ cache });


In [7]:
// 第一次运行将需要 3 秒
console.time("First run");
await graph.invoke({ messages: [{ type: "human", content: "Hello!" }] });
console.timeEnd("First run");

// 第二次运行将被缓存并立即产生
console.time("Second run");
await graph.invoke({ messages: [{ type: "human", content: "Hello!" }] });
console.timeEnd("Second run");


First run: 3.004s
Second run: 2.012ms
